# Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation
![image](https://transpchan.github.io/live3d/main.png)
This project is the official implement of [Neural Rendering with Attention: An Incremental Improvement for Anime Character Animation](https://github.com/transpchan/Live3D-v2). Watch more demos in our [HomePage](https://live3d.ml).

Press start on the left of each block, and have fun!

In [ ]:
#@title Setup Repository
#@markdown Clone git repo and install requirements.
!git clone https://github.com/transpchan/Live3D-v2.git
%cd Live3D-v2
!pip install -r requirements.txt

In [ ]:
#@title Download weights

!mkdir weights
!wget https://github.com/transpchan/Live3D-v2/releases/download/checkpoints-2.1/checkpoints.zip
!unzip checkpoints.zip -d ./weights/

In [ ]:
#@title Choose a Sample. Will download corresponding UDPs and Character Sheet. Skip if you want use your own
#@markdown Choose `double_ponytail` or `short_hair` to use sample character sheets and UDP sequences. 

#@markdown You can also choose `self_defined`, upload your own UDP sequences or pose images to `/content/CoNR/poses/`, , and upload your own character sheets to `/content/CoNR/character_sheet/character/`.

#@markdown **Caution**: The character in character sheets should be **as similar as possible to the one given in the UDP sequence**, otherwise the generated results will be uncontrollable. Please you can try using https://github.com/KurisuMakise004/MMD2UDP (notebook:https://colab.research.google.com/github/KurisuMakise004/MMD2UDP/blob/main/COLAB.ipynb) to generate your own UDP sequence.


character = 'double_ponytail' #@param ['double_ponytail', 'short_hair', 'self_defined']

In [ ]:
#@title Download corresponding UDPs and Character Sheet. Upload if you want use your own

#@markdown Click start to download sample character sheets and UDP sequences.
!rm *.zip.*
!rm -r character_sheet/
!rm -r poses/
!mkdir character_sheet/
if character == 'short_hair':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/short_hair_images.zip
  !unzip -j  short_hair_images.zip -x '__MACOSX/*'  -d character_sheet/character/ 
elif character == 'double_ponytail':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/double_ponytail_images.zip
  !unzip -j  double_ponytail_images.zip -x '__MACOSX/*' -d character_sheet/character/
else:
  print("Please upload your character sheets to /content/CoNR/character_sheet/character/")
if character == 'short_hair':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/short_hair.zip
  !unzip -j  short_hair.zip -d poses/
elif character == 'double_ponytail':
  !wget https://github.com/transpchan/Live3D-v2/releases/download/samples/double_ponytail.zip
  !unzip -j double_ponytail.zip -d poses/ 
else:
  print("Please upload your UDP sequences or poses images to /content/CoNR/poses/ .")

In [ ]:
#@title Show all character sheets
import cv2
import numpy as np
import os

path ='./character_sheet/character/'
files= os.listdir(path)
imgs = []
for file in files:
     if not os.path.isdir(file):
          img = cv2.imread(path+"/"+file, cv2.IMREAD_UNCHANGED);
          imgs.append(img)
print("Num of character sheets:", len(imgs))
imgs = np.concatenate(imgs, 1)
cv2.imwrite('character_sheet.png', imgs)

from IPython.display import Image
Image(filename='character_sheet.png') 

In [ ]:
#@title RUN!
#@markdown For sample data, this process may take about 40 minutes. You can stop earlier to get an shorter result (by clicking stop on the left).
!mkdir results
!python3 train.py --mode=test \
--dataloaders=2 \
--test_input_poses_images=./poses/ \
--test_input_person_images=./character_sheet/ \
--test_output_dir=./results/ \
--test_checkpoint_dir=./weights/ 


In [ ]:
#@title Merge generated images into a video
!ffmpeg -r 30 -y -i ./results/%d.png  -c:v qtrle output_adobe_premiere.mov 
!ffmpeg -r 30 -y -i ./results/%d.png  -c:v libvpx-vp9 output.webm 

In [39]:
#@title Show video!
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 600):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/webm;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
show_video('output.webm')